In [1]:
import pandas as pd
from underthesea import word_tokenize
import re

# Phần 1
Rút trích các review có độ dài đủ lớn

In [2]:
def process_str(input_str: str) -> str:
    if not isinstance(input_str, str):
        return ''

    output_str = bytes(input_str, 'utf-8').decode('utf-8', 'ignore').strip()
    output_str = word_tokenize(input_str, format='text')
    output_str = output_str.replace(",", " , ")
    output_str = output_str.replace("!", " ! ")
    output_str = re.sub(r"\s{2,}", " ", output_str)
    return output_str.strip().lower()


def wc_func(input_str: str) -> int:
    return len(input_str.split())


In [3]:
df = pd.read_csv('./raw.csv')
df.head()

,content,label,start
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31460 entries, 0 to 31459
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  31436 non-null  object
 1   label    31460 non-null  object
 2   start    31460 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 737.5+ KB


In [5]:
df['clean_data'] = df['content'].apply(process_str)
df['wc'] = df['clean_data'].apply(wc_func)
df.head()

,content,label,start,clean_data,wc
0,Áo bao đẹp ạ!,POS,5,áo bao đẹp ạ !,5
1,Tuyệt vời,POS,5,tuyệt_vời,1
2,2day ao khong giong trong,NEG,1,2 day ao khong_giong trong,5
3,"Mùi thơm,bôi lên da mềm da",POS,5,"mùi thơm , bôi lên da mềm da",8
4,"Vải đẹp, dày dặn",POS,5,"vải đẹp , dày dặn",5


In [6]:
df.describe()

,start,wc
count,31460.000000,31460.000000
mean,3.736840,9.037953
std,1.382483,7.576308
min,1.000000,0.000000
25%,3.000000,4.000000
50%,4.000000,7.000000
75%,5.000000,12.000000
max,5.000000,78.000000


In [7]:
new_df = df[(df['wc'] > 20) & (df['label'] != 'NEU')]
new_df.head()

,content,label,start,clean_data,wc
25,"Áo quá đẹp luôn nếu không muốn nói là đẹp, may...",POS,5,"áo quá đẹp luôn nếu không muốn nói là đẹp , ma...",30
55,"Đẹp, chất mềm mại mặc rất thích, chủ shop nhiệ...",POS,5,"đẹp , chất mềm_mại mặc rất thích , chủ_shop nh...",25
99,"Shop phục vụ rất tốt, hết mẫu mình đặt còn gọi...",POS,5,"shop phục_vụ rất tốt , hết mẫu mình đặt còn gọ...",32
114,"Chủ shop tư vấn rất nhiệt tình, đầm lên form r...",POS,5,"chủ shop tư_vấn rất nhiệt_tình , đầm lên form ...",22
124,Chất lượng sản phẩm tuyệt vời Đóng gói sản phẩ...,POS,5,chất_lượng sản_phẩm tuyệt_vời đóng_gói sản_phẩ...,23


In [8]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2215 entries, 25 to 31444
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   content     2215 non-null   object
 1   label       2215 non-null   object
 2   start       2215 non-null   int64 
 3   clean_data  2215 non-null   object
 4   wc          2215 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 103.8+ KB


In [9]:
output_df = new_df[['clean_data','label']]
output_df.rename(columns={"clean_data": "text", "label": "label"})
output_df.to_csv('clean_data.csv', index=False)

# Phần 2
Xây dựng từ điển

1. Rút trích các từ được gán nhãn tính từ
2. Rút trích các từ được gán nhãn động từ
3. gán thủ công :)

In [10]:
df = pd.read_csv('./clean_data.csv')
df.head()

,clean_data,label
0,"áo quá đẹp luôn nếu không muốn nói là đẹp , ma...",POS
1,"đẹp , chất mềm_mại mặc rất thích , chủ_shop nh...",POS
2,"shop phục_vụ rất tốt , hết mẫu mình đặt còn gọ...",POS
3,"chủ shop tư_vấn rất nhiệt_tình , đầm lên form ...",POS
4,chất_lượng sản_phẩm tuyệt_vời đóng_gói sản_phẩ...,POS


In [11]:
from underthesea import pos_tag

adj_labels = ['A', 'Ap', 'Ar', 'Ao', 'Ai']
verb_labels = ['V', 'Vi', 'Vt', 'Vs', 'Vr']

adj = {}
verb = {}

for index, row in df.iterrows():
    data = row['clean_data']
    label = row['label']
    score = 1
    if label == 'NEG':
        score = -1
    
    output = pos_tag(data)
    for item in output:
        word = item[0].strip()
        tag = item[1]
        
        # Adjective
        if tag in adj_labels:
            if word in adj.keys():
                adj[word] += score
            else:
                adj[word] = score

            continue  # skip verb section

        # Verb
        if tag in verb_labels:
            if word in verb.keys():
                verb[word] += score
            else:
                verb[word] = score

In [12]:
print(adj)

{'đẹp': 1482, 'may': 6, 'mới': 4, 'trắng': -11, 'tốt': 758, 'nhanh': 795, 'khác': -12, 'cao': 9, 'hơn': 79, 'oke': 1, 'nhiệt_tình': 33, 'hợp': 1, 'nhanh_': 5, 'ổn': 38, 'đậm': 7, 'xấu': -24, 'dài': 57, 'ok': 44, 'hài_lòng': 22, 'sạch': 5, 'dễ': 9, 'đúng': 24, 'giống': 1, 'to': 11, 'bé': 3, 'bẩn': 5, 'xanh': 3, 'gần': 2, '😐': 1, 'dằm': 1, 'chật': -5, 'nhiều': 58, 'lớn': 5, 'rộng': 29, 'kém_shop phục_vụ': -7, 'kém': -101, 'rẻ': 57, 'đắt': 8, 'mịn': 6, 'tạm': -3, 'dày': 13, 'đc': 2, 'nhanh.con': 1, 'chuẩn': 5, 'tốt_shop': 80, 'đánh_giá': 2, 'sẵn': 1, 'buồn': 2, 'shop': 0, 'nhanh_shop': 14, 'nguyên': -1, 'khô_hon': 1, 'mỏng': 22, 'đen': -9, 'nói_tóm_lại': -1, 'tệ_hại': -1, 'nhẹ': 17, 'ổn_hợp': 1, 'đỏ': 0, 'thô': -5, 'nhỏ': 24, 'gọn': 6, 'oki': 1, 'tây_nha': 1, 'tròn': 3, 'xinh': 26, 'ưng': 26, 'tren_vai': -1, 'khô': 2, 'cứng': -6, 'dễ_thương': 12, 'vui': 4, 'cực': 15, 'nóng': 3, 'mà_còn mát': 1, 'nhanh_túi': 1, 'xịn': 4, 'xám': 0, 'mềm': 22, 'khó': 6, 'tuyệt': 4, 'khó_chịu': -4, 'hỏng': -1